## 手写 MHA

import 

In [ ]:
import torch
import torch.nn as nn
import einops

class MHA(nn.module):
    def __init__(self,embed_dim,num_head,head_dim):
        self.num_head=num_head
        self.head_dim=embed_dim
        self.embed_dim=embed_dim
        self.q_linear=nn.Linear(embed_dim,embed_dim)
        self.k_linear=nn.linear(embed_dim,embed_dim)
        self.v_linear=nn.linear(embed_dim,embed_dim)
        self.o_linear=nn.linear(embed_dim,embed_dim)
    def forward(self,hidden_state,atten_mask=None):
        q,k,v=self.q_linear(hidden_state),self.k_linear(hidden_state),self.v_linear(hidden_state)
        q=einops.rearrange("b s (h d) -> b h s d",h=self.num_head)
        k=einops.rearrange("b s (h d) -> b h s d",h=self.num_head)
        v=einops.rearrange("b s (h d) -> b h s d",h=self.num_head)
        
        atten_score=torch.matmul(q,k.transpose(-1,-2))/torch.sqrt(self.head_dim)
        
        ## mask
        if atten_mask==None:
            atten_score+=atten_mask*(-1e9)# 城市一个非常大的负数
        atten_probs=torch.softmax(atten_score)
        output=torch.matmul(atten_probs,v)
        output=einops.rearrange("b h s d -> b s (h d)",h=self.num_head)
        output=self.o_linear(output)
        return output
            
